<h1> 7. Get Time corpus 2 (For BERTopic) </h1>



In [1]:
#imports
import pandas as pd
import pickle

<h4> Data after Pre-Processing 1 and 2 </h4>

Important variables: 
- df = Dataframe with all information about title, substance, class, url, text of each report
- corpus_list = List of all the report texts concatenated 

In [17]:
df = pd.read_pickle("processed_data_2.pkl")


corpus_list = []
for text in df.text:
    corpus_list += text

<h4> Seed time perception words </h4>

In [3]:
ntp_words = ['time', 'period', 'periods', 'duration', 'clock', 'temporal', 'spacetime', 'timespan', 'timespans', 'timeline', 'timelines', 'elapse', 'elapsed', 'length', 'timewise', 'velocity', 'pace', 'rate', 'tempo', 'pass', 'passing', 'passed']
ftp_words = ['quick','quicker', 'quickly', 'quickest', 'fast', 'faster', 'fastest', 'fastened', 'rapid','rapidly', 'short', 'shorter', 'shortly', 'shortest','speedy', 'speedy','speeded', 'speedier', 'hurry', 'hurried', 'swift', 'swifter', 'swiftly', 'haste', 'hasty', 'brisk', 'turbo', 'accelerate', 'acceleration', 'accelerated', 'accelerating']
stp_words = ['slow', 'slower', 'slowly', 'slows', 'slowed', 'slowest', 'slowing', 'slowdown', 'long', 'looong', 'longer', 'longer', 'longest', 'steady', 'deceleration', 'decelerate', 'decelerating', 'decelerated', 'dilatory', 'dilation', 'infinity', 'eternity', 'lengthy', 'prolonged', 'protracted', 'extended', 'unending', 'endless']
time_words = sorted(ntp_words + ftp_words + stp_words)


<h4> Modified get context window (CW) function </h4>

In some context windows, there are multiple time_words. E.g.:

*"(...)to my exceptional set and setting. later on in the future .god knows how long because time was passing so fast. i was talking to my friend t on the couch that i had yet to leave(...)"*

The modified get CW function skips over any seed words (time words), if those time words are in the context window after the intial seed word ('center_word').

In [4]:
# Modified get context window function for time_words
def modified_get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        if center_word not in time_words:
            i += 1
            pass
        else:
            context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
            for word in time_words:
                if word in context_words:
                    i += C
                    pass
            yield context_words, center_word
            i += 1


<h4> Get Time corpus 2 for BERTopic  </h4>

Format: 

|index| url | substance    | classes   | center_word | doc | 
|----------------| ------------------------- | ------ | --------- | -------- | ------ |
|5| https://erowid.org/experiences/... | 1p-lsd | Serotonergic psychedelics | time | I was talking around the ... |



In [19]:
docs = []
df2  = pd.DataFrame(columns = ["index", "url", "substance", "classes", "seed", "docs"])


def create_docs_df2(C, docs):
    for i, text in enumerate(df.text):
        url = df.loc[i, "url"]
        substance = df.loc[i, "substance"]
        classes = df.loc[i, "classes"]
        for context_words, center_word in modified_get_windows(text, C):
            if center_word in time_words: 
                doc = " ".join(context_words[0:C] + [center_word] + context_words[C:])
                if doc not in docs: #remove duplicates
                    docs.append(doc)
                    df2.loc[len(df2.index)] = [i, url, substance, classes, center_word, doc]   


C=15 #change context window size (C) here
create_docs_df2(C, docs=docs)


#save
df2.to_pickle("BERTopic_df_C=" + str(C) + ".pkl")
with open("BERTopic_docs_C=" + str(C) + ".pkl", "wb") as f:
    pickle.dump(docs, f)
